In [2]:
import os
import sys
# Here you need to have same Python version on your local machine and on worker node i.e. EC2. here both should have python3.
os.environ["PYSPARK_PYTHON"] = "/bin/python3"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_161/jre"
os.environ["SPARK_HOME"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Creating Spark Session 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('CC_Assignment').getOrCreate()
#Installing pandas
sc.install_pypi_package("pandas==0.25.1")
# Installing matplotlib
sc.install_pypi_package("matplotlib==3.1.1", "https://pypi.org/simple")
# Installing joblibspark
sc.install_pypi_package("joblibspark")
# Installing latest pip
sc.install_pypi_package("pip==22.3.1", "https://pypi.org/simple")
# Installing scikit-learn
sc.install_pypi_package("scikit-learn==1.0.2", "https://pypi.org/simple")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…




  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 30.8 MB/s eta 0:00:00

You are using pip version 9.0.1, however version 23.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

You are using pip version 9.0.1, however version 23.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

You are using pip version 9.0.1, however version 23.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

You are using pip version 22.3.1, however version 23.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: /tmp/1684914322060-0/bin/python -m pip install --upgra

In [4]:
import pyspark
import pandas as pd
import math
import scipy.stats.stats as stats
import pyspark.sql.functions as fun

from pyspark.sql.functions import col, countDistinct
from functools import reduce
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when
from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import lit, coalesce
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import DoubleType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#Reading Data from a CSV file and inferring Schema and Setting Header as True

credit_rec = spark.read.csv("s3://sqoop.oozie.ml/credit_record.csv",header = True, inferSchema = True)
credit_app = spark.read.csv("s3://sqoop.oozie.ml/application_record.csv", header = True, inferSchema = True)
def spark_shape(self):
    return (self.count(), len(self.columns))

pyspark.sql.dataframe.DataFrame.shape = spark_shape

print(credit_rec.shape())
print(credit_app.shape())

print(credit_rec.dtypes)
print(credit_app.dtypes)
credit_rec.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1048575, 3)
(438557, 18)
[('ID', 'int'), ('MONTHS_BALANCE', 'int'), ('STATUS', 'string')]
[('ID', 'int'), ('CODE_GENDER', 'string'), ('FLAG_OWN_CAR', 'string'), ('FLAG_OWN_REALTY', 'string'), ('CNT_CHILDREN', 'int'), ('AMT_INCOME_TOTAL', 'double'), ('NAME_INCOME_TYPE', 'string'), ('NAME_EDUCATION_TYPE', 'string'), ('NAME_FAMILY_STATUS', 'string'), ('NAME_HOUSING_TYPE', 'string'), ('DAYS_BIRTH', 'int'), ('DAYS_EMPLOYED', 'int'), ('FLAG_MOBIL', 'int'), ('FLAG_WORK_PHONE', 'int'), ('FLAG_PHONE', 'int'), ('FLAG_EMAIL', 'int'), ('OCCUPATION_TYPE', 'string'), ('CNT_FAM_MEMBERS', 'double')]
+-------+--------------+------+
|     ID|MONTHS_BALANCE|STATUS|
+-------+--------------+------+
|5001711|             0|     X|
|5001711|            -1|     0|
|5001711|            -2|     0|
|5001711|            -3|     0|
|5001712|             0|     C|
|5001712|            -1|     C|
|5001712|            -2|     C|
|5001712|            -3|     C|
|5001712|            -4|     C|
|5001712|            -5|

In [6]:
# Number of distinct applications 
credit_app.agg(countDistinct(col("ID")).alias("Applicants")).show()

#Number of distinct credit card customers
credit_rec.agg(countDistinct(col("ID")).alias("CC-customers")).show()

#Pivot the monthly status to get customer view
credit_rec_pivot = credit_rec.groupBy("ID").pivot("STATUS").agg(fun.count('ID'))
credit_rec_pivot.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|Applicants|
+----------+
|    438510|
+----------+

+------------+
|CC-customers|
+------------+
|       45985|
+------------+

+-------+----+----+----+----+----+----+----+----+
|     ID|   0|   1|   2|   3|   4|   5|   C|   X|
+-------+----+----+----+----+----+----+----+----+
|5003485|  14|   1|null|null|null|null|null|   2|
|5010236|   6|   1|null|null|null|null|   8|null|
|5054355|   5|null|null|null|null|null|  12|   1|
|5090249|   3|null|null|null|null|null|null|null|
|5041062|  13|null|null|null|null|null|  11|   5|
|5125237|   4|null|null|null|null|null|null|   1|
|5116308|  36|null|null|null|null|null|null|   1|
|5024813|   2|   1|null|null|null|null|  15|   1|
|5067946|   1|null|null|null|null|null|  14|   1|
|5094896|   5|null|null|null|null|null|  48|null|
|5023678|  34|null|null|null|null|null|   9|   2|
|5002975|   5|null|null|null|null|null|  19|   4|
|5099999|   4|null|null|null|null|null|  14|   1|
|5145938|  19|null|null|null|null|null|null|   1|
|5017133

In [7]:
#rename of pivoted columns
oldColumns = credit_rec_pivot.schema.names
newColumns = ["ID","stat_0","stat_1","stat_2","stat_3","stat_4","stat_5","stat_C","stat_X"]
credit_rec_pivot = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), credit_rec_pivot)
credit_rec_pivot.show()

#Delete column 
DelCols = ["stat_2","stat_3","stat_4","stat_5"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+------+------+------+------+------+------+------+
|     ID|stat_0|stat_1|stat_2|stat_3|stat_4|stat_5|stat_C|stat_X|
+-------+------+------+------+------+------+------+------+------+
|5003485|    14|     1|  null|  null|  null|  null|  null|     2|
|5010236|     6|     1|  null|  null|  null|  null|     8|  null|
|5054355|     5|  null|  null|  null|  null|  null|    12|     1|
|5090249|     3|  null|  null|  null|  null|  null|  null|  null|
|5041062|    13|  null|  null|  null|  null|  null|    11|     5|
|5125237|     4|  null|  null|  null|  null|  null|  null|     1|
|5116308|    36|  null|  null|  null|  null|  null|  null|     1|
|5024813|     2|     1|  null|  null|  null|  null|    15|     1|
|5067946|     1|  null|  null|  null|  null|  null|    14|     1|
|5094896|     5|  null|  null|  null|  null|  null|    48|  null|
|5023678|    34|  null|  null|  null|  null|  null|     9|     2|
|5002975|     5|  null|  null|  null|  null|  null|    19|     4|
|5099999| 

In [8]:
def cols_add(a,b):
    return a.__add__(b)

credit_rec_pivot = credit_rec_pivot.na.fill(0)
credit_rec_pivot = credit_rec_pivot.withColumn('total_del', reduce(cols_add, ( credit_rec_pivot[col] for col in DelCols ) ))
credit_rec_pivot.show()


#Create target variable
credit_rec_pivot = credit_rec_pivot.withColumn('target', fun.when(fun.col('total_del') > 0, 1).otherwise(0))
credit_rec_pivot.show()

credit_rec_pivot.groupby('target').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+------+------+------+------+------+------+------+---------+
|     ID|stat_0|stat_1|stat_2|stat_3|stat_4|stat_5|stat_C|stat_X|total_del|
+-------+------+------+------+------+------+------+------+------+---------+
|5003485|    14|     1|     0|     0|     0|     0|     0|     2|        0|
|5010236|     6|     1|     0|     0|     0|     0|     8|     0|        0|
|5054355|     5|     0|     0|     0|     0|     0|    12|     1|        0|
|5090249|     3|     0|     0|     0|     0|     0|     0|     0|        0|
|5041062|    13|     0|     0|     0|     0|     0|    11|     5|        0|
|5125237|     4|     0|     0|     0|     0|     0|     0|     1|        0|
|5116308|    36|     0|     0|     0|     0|     0|     0|     1|        0|
|5024813|     2|     1|     0|     0|     0|     0|    15|     1|        0|
|5067946|     1|     0|     0|     0|     0|     0|    14|     1|        0|
|5094896|     5|     0|     0|     0|     0|     0|    48|     0|        0|
|5023678|   

In [9]:
#Merging the dataframes

merged_Credit_app_records = credit_rec_pivot.join(credit_app.alias('b'), credit_rec_pivot.ID == credit_app.ID, 'inner').select('b.*', credit_rec_pivot.target)
print(merged_Credit_app_records.shape())
merged_Credit_app_records = merged_Credit_app_records.na.drop()
print(merged_Credit_app_records.shape())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(36457, 19)
(25134, 19)

### What is the proportion of females in the applicant customer base ?

In [10]:
count_gender = credit_app.groupBy('CODE_GENDER').count().orderBy('count')
count_gender = count_gender.withColumn('proportion %', fun.col('count')*100/fun.sum('count').over(Window.partitionBy()))
count_gender.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------+-----------------+
|CODE_GENDER| count|     proportion %|
+-----------+------+-----------------+
|          M|144117|32.86163486160294|
|          F|294440|67.13836513839706|
+-----------+------+-----------------+

### Is house ownership higher among male applicants vs female applicants

In [11]:
credit_app_pivoted = credit_app.crosstab('NAME_HOUSING_TYPE','CODE_GENDER')
credit_app_pivoted.show()

credit_app_pivoted = credit_app_pivoted.withColumn('prop_F', fun.col('F')/fun.sum('F').over(Window.partitionBy()))
credit_app_pivoted = credit_app_pivoted.withColumn('prop_M', fun.col('M')/fun.sum('M').over(Window.partitionBy()))

credit_app_pivoted.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------+------+------+
|NAME_HOUSING_TYPE_CODE_GENDER|     F|     M|
+-----------------------------+------+------+
|             Rented apartment|  3284|  2690|
|          Municipal apartment| 10019|  4195|
|             Office apartment|  2122|  1800|
|            House / apartment|267607|126224|
|              Co-op apartment|   862|   677|
|                 With parents| 10546|  8531|
+-----------------------------+------+------+

+-----------------------------+------+------+--------------------+--------------------+
|NAME_HOUSING_TYPE_CODE_GENDER|     F|     M|              prop_F|              prop_M|
+-----------------------------+------+------+--------------------+--------------------+
|             Rented apartment|  3284|  2690|0.011153375900013585|0.018665389926240485|
|          Municipal apartment| 10019|  4195| 0.03402730607254449|0.029108293955605515|
|             Office apartment|  2122|  1800|0.007206901236245...|0.012489851995253856|
|          

### Is there any correlation between income levels and education level?

In [12]:
# lower secondary = 0, secondary / secondary spl = 1, Incomplete higher = 2, Higher education = 3, Academic degree = 4
credit_app = credit_app.withColumn("NAME_EDUCATION_TYPE_cat", when(col("NAME_EDUCATION_TYPE")=='Lower secondary', 0).when(col("NAME_EDUCATION_TYPE")=='Secondary / secondary special', 1).when(col("NAME_EDUCATION_TYPE")=='Incomplete higher', 2).when(col("NAME_EDUCATION_TYPE")=='Higher education', 3).when(col("NAME_EDUCATION_TYPE")=='Academic degree', 4))

corl_data = credit_app.select('AMT_INCOME_TOTAL', 'NAME_EDUCATION_TYPE_cat')

col_names = corl_data.columns
features = corl_data.rdd.map(lambda row: row[0:])
corl_mat=Statistics.corr(features, method="pearson")
corl_d = pd.DataFrame(corl_mat)
corl_d.index, corl_d.columns = col_names, col_names
print(corl_d.to_string())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                         AMT_INCOME_TOTAL  NAME_EDUCATION_TYPE_cat
AMT_INCOME_TOTAL                 1.000000                 0.221403
NAME_EDUCATION_TYPE_cat          0.221403                 1.000000

### What is the average and median salary of the applicant base?

In [13]:
# Computing the mean
credit_app.select('AMT_INCOME_TOTAL').describe().show()
# Computing the median
credit_app.approxQuantile("AMT_INCOME_TOTAL", [0.5], 0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|  AMT_INCOME_TOTAL|
+-------+------------------+
|  count|            438557|
|   mean|187524.28600950394|
| stddev|110086.85306622952|
|    min|           26100.0|
|    max|         6750000.0|
+-------+------------------+

[160780.5]

### Do people owning cars have higher probability of bad customers ?

In [14]:
target_car_customers = merged_Credit_app_records.groupby('FLAG_OWN_CAR', 'target').count().orderBy('FLAG_OWN_CAR')

pivoted = merged_Credit_app_records.groupby('FLAG_OWN_CAR').pivot("target", ['1', '0']).count()

pivoted.show()

row_count = sum(col(x) for x in ['1', '0'])

adjusted = [(col(y) / row_count).alias(y) for y in ['1', '0']]

pivoted.select(col("FLAG_OWN_CAR"), *adjusted).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+---+-----+
|FLAG_OWN_CAR|  1|    0|
+------------+---+-----+
|           Y|177|10339|
|           N|245|14373|
+------------+---+-----+

+------------+--------------------+------------------+
|FLAG_OWN_CAR|                   1|                 0|
+------------+--------------------+------------------+
|           Y| 0.01683149486496767|0.9831685051350323|
|           N|0.016760158708441646|0.9832398412915584|
+------------+--------------------+------------------+

### Do people living on rent have a higher proportion of bad customers as compared to the rest of the population

In [15]:
merged_Credit_app_records.select("NAME_HOUSING_TYPE").distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|  NAME_HOUSING_TYPE|
+-------------------+
|Municipal apartment|
|   Rented apartment|
|  House / apartment|
|    Co-op apartment|
|   Office apartment|
|       With parents|
+-------------------+

In [16]:
merged_Credit_app_records = merged_Credit_app_records.withColumn('RENT_HOUSING_TYPE', fun.when(fun.col('NAME_HOUSING_TYPE') == 'Rented apartment', 'Rent').otherwise('Not Rent'))

target_rent_customers = merged_Credit_app_records.groupby('RENT_HOUSING_TYPE', 'target').count().orderBy('RENT_HOUSING_TYPE')

pivoted_data = merged_Credit_app_records.groupby('RENT_HOUSING_TYPE').pivot("target", ['1', '0']).count()

pivoted_data.show()

pivoted_data.select(col("RENT_HOUSING_TYPE"), *adjusted).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+---+-----+
|RENT_HOUSING_TYPE|  1|    0|
+-----------------+---+-----+
|         Not Rent|416|24279|
|             Rent|  6|  433|
+-----------------+---+-----+

+-----------------+--------------------+------------------+
|RENT_HOUSING_TYPE|                   1|                 0|
+-----------------+--------------------+------------------+
|         Not Rent|0.016845515286495243|0.9831544847135048|
|             Rent| 0.01366742596810934|0.9863325740318907|
+-----------------+--------------------+------------------+

### Do single people have a high proportion of bad customers as compared to married customers ?

In [17]:
merged_Credit_app_records.select("NAME_FAMILY_STATUS").distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|  NAME_FAMILY_STATUS|
+--------------------+
|           Separated|
|             Married|
|Single / not married|
|      Civil marriage|
|               Widow|
+--------------------+

In [18]:
merged_Credit_app_records = merged_Credit_app_records.withColumn('MARRIED_STATUS', fun.when(fun.col('NAME_FAMILY_STATUS') == 'Married', 'Married').otherwise('Unmarried'))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
target_married_customers = merged_Credit_app_records.groupby('MARRIED_STATUS', 'target').count().orderBy('MARRIED_STATUS')

pivoted_data = merged_Credit_app_records.groupby('MARRIED_STATUS').pivot("target", ['1', '0']).count()

pivoted_data.show()

pivoted_data.select(col("MARRIED_STATUS"), *adjusted).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+---+-----+
|MARRIED_STATUS|  1|    0|
+--------------+---+-----+
|       Married|277|17232|
|     Unmarried|145| 7480|
+--------------+---+-----+

+--------------+-------------------+------------------+
|MARRIED_STATUS|                  1|                 0|
+--------------+-------------------+------------------+
|       Married|0.01582043520475184|0.9841795647952482|
|     Unmarried|0.01901639344262295| 0.980983606557377|
+--------------+-------------------+------------------+

In [20]:
# WOE & IV calculation by creating a class

class WOE_IV(object):
    def __init__(self, df: DataFrame, cols_to_woe: [str], label_column: str, good_label: str):
        self.df = df
        self.cols_to_woe = cols_to_woe
        self.label_column = label_column
        self.good_label = good_label
        self.fit_data = {}

    def fit(self):
        for col_to_woe in self.cols_to_woe:
            total_good = self.compute_total_amount_of_good()
            total_bad = self.compute_total_amount_of_bad()
            
            woe_df = self.df.select(col_to_woe)
            categories = woe_df.distinct().collect()
            for category_row in categories:
                category = category_row[col_to_woe]
                good_amount = self.compute_good_amount(col_to_woe, category)
                bad_amount = self.compute_bad_amount(col_to_woe, category)

                good_amount = good_amount if good_amount != 0 else 0.5
                bad_amount = bad_amount if bad_amount != 0 else 0.5

                good_dist = good_amount / total_good
                bad_dist = bad_amount / total_bad

                self.build_fit_data(col_to_woe, category, good_dist, bad_dist)

    def transform(self, df: DataFrame):
        def _encode_woe(col_to_woe_):
            return F.coalesce(
                *[F.when(F.col(col_to_woe_) == category, F.lit(woe_iv['woe']))
                  for category, woe_iv in self.fit_data[col_to_woe_].items()]
            )

        for col_to_woe, woe_info in self.fit_data.items():
            df = df.withColumn(col_to_woe + '_woe', _encode_woe(col_to_woe))
        return df

    def compute_total_amount_of_good(self):
        return self.df.select(self.label_column).filter(F.col(self.label_column) == self.good_label).count()

    def compute_total_amount_of_bad(self):
        return self.df.select(self.label_column).filter(F.col(self.label_column) != self.good_label).count()

    def compute_good_amount(self, col_to_woe: str, category: str):
        return self.df.select(col_to_woe, self.label_column)\
                      .filter(
                            (F.col(col_to_woe) == category) & (F.col(self.label_column) == self.good_label)
                      ).count()

    def compute_bad_amount(self, col_to_woe: str, category: str):
        return self.df.select(col_to_woe, self.label_column)\
                      .filter(
                            (F.col(col_to_woe) == category) & (F.col(self.label_column) != self.good_label)
                      ).count()

    def build_fit_data(self, col_to_woe, category, good_dist, bad_dist):
        woe_info = {
            category: {
                'woe': math.log(good_dist / bad_dist),
                'iv': (good_dist - bad_dist) * math.log(good_dist / bad_dist)
            }
        }

        if col_to_woe not in self.fit_data:
            self.fit_data[col_to_woe] = woe_info
        else:
            self.fit_data[col_to_woe].update(woe_info)

    def compute_iv(self):
        iv_dict = {}

        for woe_col, categories in self.fit_data.items():
            iv_dict[woe_col] = 0
            for category, woe_iv in categories.items():
                iv_dict[woe_col] += woe_iv['iv']
        return iv_dict

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# bins for continous variables

spike_columns = ['CNT_CHILDREN','AMT_INCOME_TOTAL','DAYS_BIRTH','DAYS_EMPLOYED','CNT_FAM_MEMBERS']

max_bucket_size = 5

for x in spike_columns:
    rt = 0
    num = max_bucket_size
    while ((np.abs(rt) < 1) & (num>1)):
        discretizer = QuantileDiscretizer(numBuckets=num, inputCol=x, outputCol=x + "bucket")
        d = discretizer.fit(merged_Credit_app_records).transform(merged_Credit_app_records)
        d1 = d.groupby(x+ "bucket").agg(F.mean(x).alias("X1"),F.mean("target").alias("Y1")).orderBy("X1")
        a1 = d1.select("X1").rdd.flatMap(lambda x: x).collect()
        b1 = d1.select("Y1").rdd.flatMap(lambda x: x).collect()
        rt, p = stats.spearmanr(a1, b1)
        if(rt == 1):
            print(x+"bucket")
            merged_Credit_app_records = discretizer.fit(merged_Credit_app_records).transform(merged_Credit_app_records)
        num = num-1
        
print(merged_Credit_app_records.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

AMT_INCOME_TOTALbucket
DAYS_EMPLOYEDbucket
['ID', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'target', 'RENT_HOUSING_TYPE', 'MARRIED_STATUS', 'AMT_INCOME_TOTALbucket', 'DAYS_EMPLOYEDbucket']

In [22]:
# Calculating WOE and IV for all the categorical variables (including bin variables from continous variables)


columns_to_woe = ['CODE_GENDER', 'FLAG_OWN_CAR','FLAG_OWN_REALTY','FLAG_OWN_REALTY','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE',
               'NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','FLAG_MOBIL','FLAG_WORK_PHONE','FLAG_PHONE','FLAG_EMAIL','OCCUPATION_TYPE',
               'AMT_INCOME_TOTALbucket','DAYS_EMPLOYEDbucket']

woe = WOE_IV(merged_Credit_app_records, columns_to_woe, 'target', 1)

# woe encoding
woe.fit()
encoded_df = woe.transform(merged_Credit_app_records)

# information value
ivs = woe.compute_iv()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-22:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1798



In [23]:
# print WOE for each variable & category
print(woe.fit_data)
print(ivs)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'CODE_GENDER': {'M': {'woe': 0.18847512261896812, 'iv': 0.014730334586125406}, 'F': {'woe': -0.13400455891611948, 'iv': 0.010473169941325407}}, 'FLAG_OWN_CAR': {'Y': {'woe': 0.002510221549541952, 'iv': 2.639611117445198e-06}, 'N': {'woe': -0.0018095944440989517, 'iv': 1.902870132551552e-06}}, 'FLAG_OWN_REALTY': {'Y': {'woe': -0.13077665160154833, 'iv': 0.010521107917778397}, 'N': {'woe': 0.21030941552994442, 'iv': 0.016919595583905052}}, 'NAME_INCOME_TYPE': {'Commercial associate': {'woe': 0.005114514731444934, 'iv': 7.357550450491796e-06}, 'Working': {'woe': -0.005027780864046354, 'iv': 1.567375175195009e-05}, 'Pensioner': {'woe': 7.328135458583888, 'iv': 0.2255999779354912}, 'Student': {'woe': 1.0743066470084148, 'iv': 0.0008381444170551353}, 'State servant': {'woe': -0.34921705587983176, 'iv': 0.010030267766525755}}, 'NAME_EDUCATION_TYPE': {'Higher education': {'woe': 0.0678324910762806, 'iv': 0.0013493563969528277}, 'Academic degree': {'woe': 0.7378344103872019, 'iv': 0.0004562087

In [24]:
#writing the contents to a file for check point

encoded_df.write.format("com.databricks.spark.csv").mode('overwrite').save('encodedfile.csv',header = 'true')
encoded_df_saved = spark.read.csv("encodedfile.csv", header=True, inferSchema=True)
print(encoded_df.shape())
print(encoded_df_saved.shape())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-24:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 2197



(25134, 37)
(25134, 37)

In [26]:
#loading the data from the saved file 

encoded_df_saved = spark.read.csv("encodedfile.csv", header=True, inferSchema=True)
print(encoded_df_saved.shape())
encoded_df_saved.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(25134, 37)
['ID', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'target', 'RENT_HOUSING_TYPE', 'MARRIED_STATUS', 'AMT_INCOME_TOTALbucket', 'DAYS_EMPLOYEDbucket', 'CODE_GENDER_woe', 'FLAG_OWN_CAR_woe', 'FLAG_OWN_REALTY_woe', 'NAME_INCOME_TYPE_woe', 'NAME_EDUCATION_TYPE_woe', 'NAME_FAMILY_STATUS_woe', 'NAME_HOUSING_TYPE_woe', 'FLAG_MOBIL_woe', 'FLAG_WORK_PHONE_woe', 'FLAG_PHONE_woe', 'FLAG_EMAIL_woe', 'OCCUPATION_TYPE_woe', 'AMT_INCOME_TOTALbucket_woe', 'DAYS_EMPLOYEDbucket_woe']

In [27]:
#splitting in test and train data
train, test = encoded_df_saved.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

#Specifying feature list to be used for logistic regression
features_list = [ 'CODE_GENDER_woe','FLAG_OWN_CAR_woe','FLAG_OWN_REALTY_woe','NAME_EDUCATION_TYPE_woe','NAME_FAMILY_STATUS_woe',
'NAME_HOUSING_TYPE_woe','FLAG_MOBIL_woe','FLAG_WORK_PHONE_woe','FLAG_PHONE_woe','FLAG_EMAIL_woe','OCCUPATION_TYPE_woe',
'AMT_INCOME_TOTALbucket_woe','DAYS_EMPLOYEDbucket_woe']

#Creating feature vector column with all the input variables
assembler = VectorAssembler(inputCols=features_list, outputCol="features")
train = assembler.transform(train)
test = assembler.transform(test)

#Building logistic regression model and predict on test & train dataset using the same model 
lr = LogisticRegression(labelCol="target", featuresCol= "features",maxIter=10)
model=lr.fit(train)
predict_train=model.transform(train)
predict_test=model.transform(test)
predict_test.select("target","prediction","probability").show(truncate=False)

#Creating a probability column for target probability 
secondelement=udf(lambda v:float(v[1]),FloatType())
predict_train = predict_train.withColumn('prob_int', secondelement('probability'))
predict_test = predict_test.withColumn('prob_int', secondelement('probability'))
predict_test.select("target","prediction","probability","prob_int").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training Dataset Count: 17541
Test Dataset Count: 7593
+------+----------+-----------------------------------------+
|target|prediction|probability                              |
+------+----------+-----------------------------------------+
|0     |0.0       |[0.9846176278381278,0.015382372161872151]|
|0     |0.0       |[0.9846176278381278,0.015382372161872151]|
|0     |0.0       |[0.986773310087667,0.01322668991233313]  |
|0     |0.0       |[0.9863673224127191,0.013632677587280992]|
|0     |0.0       |[0.9863673224127191,0.013632677587280992]|
|0     |0.0       |[0.9872500834417228,0.012749916558277178]|
|0     |0.0       |[0.9872500834417228,0.012749916558277178]|
|0     |0.0       |[0.990614606918368,0.00938539308163209]  |
|0     |0.0       |[0.990614606918368,0.00938539308163209]  |
|0     |0.0       |[0.990614606918368,0.00938539308163209]  |
|0     |0.0       |[0.990614606918368,0.00938539308163209]  |
|0     |0.0       |[0.990614606918368,0.00938539308163209]  |
|0     |0.0    

In [28]:
# AUC ROC for training set
trainingSummary = model.summary
roc = trainingSummary.roc.toPandas()
plt.plot(roc['FPR'],roc['TPR'])
plt.ylabel('False Positive Rate')
plt.xlabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()
print('Training set areaUnderROC: ' + str(trainingSummary.areaUnderROC))


# AUC ROC for test set
preds = predict_test.select('target','prob_int').rdd.map(lambda row: (float(row['prob_int']), float(row['target']))).collect()
lr_probs, y_test = zip(*preds)
fpr, tpr, thresholds = roc_curve(y_test, lr_probs, pos_label = 1)

ns_probs = [0 for _ in range(len(y_test))]
# calculate scores
ns_auc = roc_auc_score(y_test, ns_probs)
lr_auc = roc_auc_score(y_test, lr_probs)

# summarize scores
print('Testing set areaUnderROC=%.3f' % (lr_auc))
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training set areaUnderROC: 0.6454687677330996
Testing set areaUnderROC=0.621

In [29]:
#calculating KS

#Creating deciles using the predcited probability column 
discretizer = QuantileDiscretizer(numBuckets=10, inputCol="prob_int", outputCol="PROB_BUCKET")
predict_train = discretizer.fit(predict_train).transform(predict_train)
predict_test = discretizer.fit(predict_test).transform(predict_test)

#Creating #Good & # bad for each decile 
ks_train = predict_train.groupby('PROB_BUCKET').agg(F.count("ID").alias("Total"), 
                                                    F.max("prob_int").alias("max_prob"), 
                                                    F.min("prob_int").alias("min_prob"),  
                                                    F.sum("target").alias("bads") ,
                                                    F.count(F.when(col("target") == 0,1)).alias("goods")).sort(col('PROB_BUCKET').desc())

ks_test = predict_test.groupby('PROB_BUCKET').agg(F.count("ID").alias("Total"), 
                                                  F.max("prob_int").alias("max_prob"), 
                                                  F.min("prob_int").alias("min_prob"),  
                                                  F.sum("target").alias("bads") ,
                                                  F.count(F.when(col("target") == 0,1)).alias("goods")).sort(col('PROB_BUCKET').desc())

#Creating %cumulative good,%cumulative bad & KS 
ks_train = ks_train.withColumn('%cumuBad', F.sum('bads').over(Window.orderBy(F.col('PROB_BUCKET').desc()).rowsBetween(Window.unboundedPreceding, 0)
)/ F.sum('bads').over(Window.partitionBy()))
ks_train = ks_train.withColumn('%cumuGood', F.sum('goods').over(Window.orderBy(F.col('PROB_BUCKET').desc()).rowsBetween(Window.unboundedPreceding, 0)
)/ F.sum('goods').over(Window.partitionBy()))
ks_train = ks_train.withColumn('%diff', F.col('%cumuBad') -  F.col('%cumuGood'))
ks_train = ks_train.withColumn('max_diff', F.when(F.col('%diff') ==F.max('%diff').over(Window.partitionBy()) ,'<----').otherwise(' '))

ks_test = ks_test.withColumn('%cumuBad', F.sum('bads').over(Window.orderBy(F.col('PROB_BUCKET').desc()).rowsBetween(Window.unboundedPreceding, 0)
)/ F.sum('bads').over(Window.partitionBy()))
ks_test = ks_test.withColumn('%cumuGood', F.sum('goods').over(Window.orderBy(F.col('PROB_BUCKET').desc()).rowsBetween(Window.unboundedPreceding, 0)
)/ F.sum('goods').over(Window.partitionBy()))
ks_test = ks_test.withColumn('%diff', F.col('%cumuBad') -  F.col('%cumuGood'))
ks_test = ks_test.withColumn('max_diff', F.when(F.col('%diff') ==F.max('%diff').over(Window.partitionBy()) ,'<----').otherwise(' '))

ks_train.sort(col('PROB_BUCKET').desc()).show()
ks_test.sort(col('PROB_BUCKET').desc()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----+-----------+------------+----+-----+-------------------+-------------------+-------------------+--------+
|PROB_BUCKET|Total|   max_prob|    min_prob|bads|goods|           %cumuBad|          %cumuGood|              %diff|max_diff|
+-----------+-----+-----------+------------+----+-----+-------------------+-------------------+-------------------+--------+
|        9.0| 1765| 0.12203759| 0.028323418|  64| 1701| 0.2229965156794425|0.09858583516865654|0.12441068051078596|        |
|        8.0| 1748|0.028291991| 0.021746911|  49| 1699|0.39372822299651566|0.19705575518720297| 0.1966724678093127|        |
|        7.0| 1762|0.021745775| 0.018268326|  27| 1735| 0.4878048780487805|0.29761214790773155|0.19019273014104893|        |
|        6.0| 1738|0.018267384| 0.015872356|  34| 1704| 0.6062717770034843|0.39637185580155326|0.20989992120193107|   <----|
|        5.0| 1763|0.015871823| 0.014023574|  25| 1738| 0.6933797909407665|  0.497102121247247| 0.1962776696935195|        |


In [ ]:
{
 'DAYS_EMPLOYEDbucket': 0.09953102010566758   
 'OCCUPATION_TYPE': 0.09005135568358584,
 'NAME_FAMILY_STATUS': 0.04313711542312794
 'FLAG_OWN_REALTY': 0.02744070350168345, 
 'CODE_GENDER': 0.025203504527450814, 
 'NAME_EDUCATION_TYPE': 0.010895022914241295,
 'NAME_INCOME_TYPE': 0.23649142142127452, 
 'NAME_HOUSING_TYPE': 0.007327502688022734,
 'AMT_INCOME_TOTALbucket': 0.0022134026915242517,

 'FLAG_OWN_CAR': 4.5424812499967504e-06,   
 'FLAG_MOBIL': 0.0, 
 'FLAG_WORK_PHONE': 0.00204242979514846, 
 'FLAG_PHONE': 0.0005480495762639285, 
 'FLAG_EMAIL': 1.7343581493999582e-05}

In [30]:
#splitting in test and train data
train, test = encoded_df_saved.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

#Specifying feature list to be used for logistic regression
features_list = ['AMT_INCOME_TOTALbucket_woe','CODE_GENDER_woe','DAYS_EMPLOYEDbucket_woe','FLAG_OWN_REALTY_woe',
                 'NAME_EDUCATION_TYPE_woe','NAME_FAMILY_STATUS_woe','NAME_HOUSING_TYPE_woe','OCCUPATION_TYPE_woe',
                 'NAME_INCOME_TYPE_woe']

#Creating feature vector column with all the input variables
assembler = VectorAssembler(inputCols=features_list, outputCol="features")
train = assembler.transform(train)
test = assembler.transform(test)

#Building logistic regression model and predict on test & train dataset using the same model 
lr = LogisticRegression(labelCol="target", featuresCol= "features",maxIter=10)
model=lr.fit(train)
predict_train=model.transform(train)
predict_test=model.transform(test)
#predict_test.select("target","prediction","probability").show(truncate=False)

#Creating a probability column for target probability 
secondelement=udf(lambda v:float(v[1]),FloatType())
predict_train = predict_train.withColumn('prob_int', secondelement('probability'))
predict_test = predict_test.withColumn('prob_int', secondelement('probability'))

# AUC ROC for training set
trainingSummary = model.summary
roc = trainingSummary.roc.toPandas()
plt.plot(roc['FPR'],roc['TPR'])
plt.ylabel('False Positive Rate')
plt.xlabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()
print('Training set areaUnderROC: ' + str(trainingSummary.areaUnderROC))


# AUC ROC for test set
preds = predict_test.select('target','prob_int').rdd.map(lambda row: (float(row['prob_int']), float(row['target']))).collect()
lr_probs, y_test = zip(*preds)
fpr, tpr, thresholds = roc_curve(y_test, lr_probs, pos_label = 1)

ns_probs = [0 for _ in range(len(y_test))]
# calculate scores
ns_auc = roc_auc_score(y_test, ns_probs)
lr_auc = roc_auc_score(y_test, lr_probs)

# summarize scores
print('Testing set areaUnderROC=%.3f' % (lr_auc))
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()


#calculating KS

#Creating deciles using the predcited probability column 
discretizer = QuantileDiscretizer(numBuckets=10, inputCol="prob_int", outputCol="PROB_BUCKET")
predict_train = discretizer.fit(predict_train).transform(predict_train)
predict_test = discretizer.fit(predict_test).transform(predict_test)

#Creating #Good & # bad for each decile 
ks_train = predict_train.groupby('PROB_BUCKET').agg(F.count("ID").alias("Total"), 
                                                    F.max("prob_int").alias("max_prob"), 
                                                    F.min("prob_int").alias("min_prob"),  
                                                    F.sum("target").alias("bads") ,
                                                    F.count(F.when(col("target") == 0,1)).alias("goods")).sort(col('PROB_BUCKET').desc())

ks_test = predict_test.groupby('PROB_BUCKET').agg(F.count("ID").alias("Total"), 
                                                  F.max("prob_int").alias("max_prob"), 
                                                  F.min("prob_int").alias("min_prob"),  
                                                  F.sum("target").alias("bads") ,
                                                  F.count(F.when(col("target") == 0,1)).alias("goods")).sort(col('PROB_BUCKET').desc())

#Creating %cumulative good,%cumulative bad & KS 
ks_train = ks_train.withColumn('%cumuBad', F.sum('bads').over(Window.orderBy(F.col('PROB_BUCKET').desc()).rowsBetween(Window.unboundedPreceding, 0)
)/ F.sum('bads').over(Window.partitionBy()))
ks_train = ks_train.withColumn('%cumuGood', F.sum('goods').over(Window.orderBy(F.col('PROB_BUCKET').desc()).rowsBetween(Window.unboundedPreceding, 0)
)/ F.sum('goods').over(Window.partitionBy()))
ks_train = ks_train.withColumn('%diff', F.col('%cumuBad') -  F.col('%cumuGood'))
ks_train = ks_train.withColumn('max_diff', F.when(F.col('%diff') ==F.max('%diff').over(Window.partitionBy()) ,'<----').otherwise(' '))

ks_test = ks_test.withColumn('%cumuBad', F.sum('bads').over(Window.orderBy(F.col('PROB_BUCKET').desc()).rowsBetween(Window.unboundedPreceding, 0)
)/ F.sum('bads').over(Window.partitionBy()))
ks_test = ks_test.withColumn('%cumuGood', F.sum('goods').over(Window.orderBy(F.col('PROB_BUCKET').desc()).rowsBetween(Window.unboundedPreceding, 0)
)/ F.sum('goods').over(Window.partitionBy()))
ks_test = ks_test.withColumn('%diff', F.col('%cumuBad') -  F.col('%cumuGood'))
ks_test = ks_test.withColumn('max_diff', F.when(F.col('%diff') ==F.max('%diff').over(Window.partitionBy()) ,'<----').otherwise(' '))

ks_train.sort(col('PROB_BUCKET').desc()).show()
ks_test.sort(col('PROB_BUCKET').desc()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-30:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 2290



Training Dataset Count: 17541
Test Dataset Count: 7593
Training set areaUnderROC: 0.6521478229155769
Testing set areaUnderROC=0.649
+-----------+-----+------------+------------+----+-----+-------------------+-------------------+-------------------+--------+
|PROB_BUCKET|Total|    max_prob|    min_prob|bads|goods|           %cumuBad|          %cumuGood|              %diff|max_diff|
+-----------+-----+------------+------------+----+-----+-------------------+-------------------+-------------------+--------+
|        9.0| 1766|   0.9828491| 0.027494008|  63| 1703|0.21951219512195122|0.09870175031876666|0.12081044480318456|        |
|        8.0| 1739| 0.027488615| 0.021537388|  46| 1693| 0.3797909407665505|0.19682392488698272| 0.1829670158795678|        |
|        7.0| 1770| 0.021510385|  0.01797418|  37| 1733| 0.5087108013937283|0.29726440245740116| 0.2114463989363271|        |
|        6.0| 1735| 0.017968645| 0.015870653|  37| 1698| 0.6376306620209059|0.39567636490089253|0.24195429712001

In [32]:
#Creating prediction classs based on the prob cut off by KS value

min_prob_max_ks = ks_test.where(col("max_diff") == '<----').select(col("min_prob")).collect()[0]
predict_test = predict_test.withColumn('pred_class', F.when(F.col('prob_int') > round(min_prob_max_ks[0],4) ,1).otherwise(0))
predict_test.groupBy('pred_class','target').count().show()

#Generating evaluation metrics
results = predict_test.select(['pred_class', 'target'])
results = results.withColumn("target", results["target"].cast(DoubleType()))
results = results.withColumn("pred_class", results["pred_class"].cast(DoubleType()))

predictionAndLabels=results.rdd
metrics = MulticlassMetrics(predictionAndLabels)
metrics1 = BinaryClassificationMetrics(predictionAndLabels)

cm=metrics.confusionMatrix().toArray()
print(cm)
# Overall statistics
precision = metrics.precision(1)
recall = metrics.recall(1)
f1Score = metrics.fMeasure()
print("Summary Stats")
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1 Score = %s" % f1Score)
# Area under precision-recall curve
print("Area under PR = %s" % metrics1.areaUnderPR)
# Area under ROC curve
print("Area under ROC = %s" % metrics1.areaUnderROC)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-32:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 2398



+----------+------+-----+
|pred_class|target|count|
+----------+------+-----+
|         1|     1|   90|
|         0|     0| 4497|
|         0|     1|   45|
|         1|     0| 2961|
+----------+------+-----+

[[4497. 2961.]
 [  45.   90.]]
Summary Stats
Precision = 0.029498525073746312
Recall = 0.6666666666666666
F1 Score = 0.6041090478071909
Area under PR = 0.027545359858307625
Area under ROC = 0.6348216680075086